In [1]:
import os
from pathlib import Path
from PIL import Image
from typing import Any, Literal

from prettytable import PrettyTable
import skimage as ski
import lightning as L
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchmetrics as tm
from torchvision.models import get_model, get_weight
from torchvision.datasets import DatasetFolder
from torchvision.transforms import v2

# Setup

In [2]:
DATASET_PATHS = {
    'fog-detection': Path('./datasets/fog-detection-dataset-prepared'),
    'fog-or-smog': Path('./datasets/fog-or-smog-detection-dataset-prepared'),
    'foggy-cityscapes': Path('./datasets/foggy-cityscapes-image-dataset-prepared')
}

In [3]:
def load_image(path: str):
    img = ski.io.imread(path)
    if img.ndim == 2:  # Handle grayscale
        img = ski.color.gray2rgb(img)
    if img.shape[-1] == 4:  # Handle RGBA
        img = ski.color.rgba2rgb(img)
    img = ski.util.img_as_ubyte(img)
    img = img.squeeze()
    
    return Image.fromarray(img)

train_transforms = v2.Compose([
    v2.ToImage(),
    v2.Grayscale(1),
    v2.RandomResizedCrop(size=(224,224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5], std=[0.25]),
    v2.RandomVerticalFlip(p=1.0)
])

test_transforms = v2.Compose([
    v2.ToImage(),
    v2.Grayscale(1),
    v2.Resize(size=(224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5], std=[0.25]),
    v2.RandomVerticalFlip(p=1.0)
])

def get_dataloader(
    path: Path | str,
    transform: Any | None = None,
    batch_size: int = 32,
    shuffle: bool = False,
    num_workers: int = 0,
    pin_memory: bool = True,
):
    return DataLoader(
        dataset=DatasetFolder(
            root=path,
            loader=load_image,
            extensions=[".jpg", ".png", ".jpeg"],
            transform=transform,
        ),
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=True if num_workers > 0 else False,
    )

In [4]:
dl = get_dataloader(DATASET_PATHS['fog-detection'], train_transforms, batch_size=2)
x, y = next(iter(dl))
x.shape, y.shape

(torch.Size([2, 1, 224, 224]), torch.Size([2]))

In [5]:
class LSTMClassifier(L.LightningModule):
    def __init__(
        self,
        image_width: int,
        image_height: int,
        image_channels: int,
        hidden_size: int,
        num_lstm_layers: int,
        num_classes: int,
        dropout: float = 0.0,
        loss: nn.Module | None = None,
        learning_rate: float= 1e-4,
        optimizer_name: Literal['adam', 'adamw'] = 'adam',
    ):
        super().__init__()
        self.save_hyperparameters()

        self.learning_rate = learning_rate
        self.optimizer_name = optimizer_name

        self.image_height = image_height
        self.image_width = image_width
        self.image_channels = image_channels
        self.hiddes_size = hidden_size
        self.num_lst_layers = num_lstm_layers
        self.lstm_input_size = self.image_channels * self.image_width
        self.num_classes = num_classes
        self.dropout = dropout
        self.model = nn.LSTM(
            input_size=self.lstm_input_size,
            hidden_size=self.hiddes_size,
            num_layers=self.num_lst_layers,
            batch_first=True,
            dropout=self.dropout
        )
        self.fc = nn.Linear(self.hiddes_size, self.num_classes)

        if loss is not None:
            self.criterion= loss
        else:
            self.criterion = nn.CrossEntropyLoss()

        task = "multiclass" if num_classes > 2 else "binary"
        
        self.train_metrics = tm.MetricCollection({
            "accuracy": tm.classification.Accuracy(task=task, num_classes=num_classes),
            "f1": tm.classification.F1Score(task=task, num_classes=num_classes),
            "precision": tm.classification.Precision(task=task, num_classes=num_classes),
            "recall": tm.classification.Recall(task=task, num_classes=num_classes),
        }, prefix="train_")
        self.validation_metrics = self.train_metrics.clone(prefix="val_")
        self.test_metrics = self.train_metrics.clone(prefix="test_")

    def forward(self, x):
        batch_size = x.size(0)

        x = x.permute(0, 2, 1, 3)
        x = x.reshape(batch_size, self.image_height, self.lstm_input_size)

        lstm_out, (hn, cn) = self.model(x)
        last_time_step_out= lstm_out[:, -1, :]
        
        return self.fc(last_time_step_out)
    
    def _common_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = self.criterion(logits, labels)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, labels
    
    def training_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)

        self.log("train_loss", loss)
        self.log_dict(self.train_metrics(preds, labels))

        return loss
    
    def on_train_epoch_end(self):
        self.train_metrics.reset()

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.validation_metrics.update(preds, labels)
        self.log("val_loss", loss)
        return loss
    
    def on_validation_epoch_end(self):
        self.log_dict(self.validation_metrics.compute())
        self.validation_metrics.reset()
    

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.test_metrics.update(preds, labels)
        return loss
    
    def on_test_epoch_end(self):
        self.log_dict(self.test_metrics.compute())
        self.test_metrics.reset()

    def configure_optimizers(self):
        if self.optimizer_name == 'adam':
            optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        elif self.optimizer_name == 'adamw':
            optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer_name}.")
        return optimizer

# Training

## Normalized

In [6]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-GRAYSCALE')
MODEL_NAME = "LSTM"
DATASET = 'fog-detection'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 1
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-GRAYSCALE/LSTM-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 313 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
313 K     Trainable params
0         Non-trainable pa

Epoch 18: 100%|██████████| 12/12 [00:07<00:00,  1.57it/s, v_num=1]         


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.10it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6415094137191772     │
│          test_f1          │    0.7564102411270142     │
│      test_precision       │    0.6483516693115234     │
│        test_recall        │    0.9076923131942749     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5775862336158752     │
│          test_f1          │    0.7021276354789734     │
│      test_precision       │    0.5526315569877625     │
│        test_recall        │    0.9624999761581421     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:13<00:00,  0.53it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5404040217399597     │
│          test_f1          │    0.6513410210609436     │
│      test_precision       │    0.5246913433074951     │
│        test_recall        │    0.8585858345031738     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6415,0.7564,0.6484,0.9077
fog-or-smog,0.5776,0.7021,0.5526,0.9625
foggy-cityscapes,0.5404,0.6513,0.5247,0.8586


In [7]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-GRAYSCALE')
MODEL_NAME = "LSTM"
DATASET = 'fog-or-smog'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 1
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-GRAYSCALE/LSTM-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 313 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
313 K     Trainable params
0         Non-trainable para

Epoch 6: 100%|██████████| 52/52 [00:07<00:00,  7.14it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.15it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6226415038108826     │
│          test_f1          │    0.6610169410705566     │
│      test_precision       │    0.7358490824699402     │
│        test_recall        │    0.6000000238418579     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7715517282485962     │
│          test_f1          │    0.7953668236732483     │
│      test_precision       │    0.7410072088241577     │
│        test_recall        │    0.8583333492279053     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5909090638160706     │
│          test_f1          │    0.5090909004211426     │
│      test_precision       │    0.6363636255264282     │
│        test_recall        │    0.42424243688583374    │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6226,0.661,0.7358,0.6
fog-or-smog,0.7716,0.7954,0.741,0.8583
foggy-cityscapes,0.5909,0.5091,0.6364,0.4242


In [8]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-GRAYSCALE')
MODEL_NAME = "LSTM"
DATASET = 'foggy-cityscapes'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 1
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-GRAYSCALE/LSTM-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 313 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
313 K     Trainable params
0         Non-trainable

Epoch 7: 100%|██████████| 22/22 [00:15<00:00,  1.43it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6415094137191772     │
│          test_f1          │    0.7121211886405945     │
│      test_precision       │    0.7014925479888916     │
│        test_recall        │    0.7230769395828247     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.756465494632721     │
│          test_f1          │    0.8007054924964905     │
│      test_precision       │    0.6941896080970764     │
│        test_recall        │    0.9458333253860474     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.55it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6212121248245239     │
│          test_f1          │    0.6987951993942261     │
│      test_precision       │    0.5799999833106995     │
│        test_recall        │    0.8787878751754761     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6415,0.7121,0.7015,0.7231
fog-or-smog,0.7565,0.8007,0.6942,0.9458
foggy-cityscapes,0.6212,0.6988,0.58,0.8788


## No normalization

In [9]:
train_transforms = v2.Compose([
    v2.ToImage(),
    v2.Grayscale(1),
    v2.RandomResizedCrop(size=(224,224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize(mean=[0.5], std=[0.25]),
    v2.RandomVerticalFlip(p=1.0)
])

test_transforms = v2.Compose([
    v2.ToImage(),
    v2.Grayscale(1),
    v2.Resize(size=(224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    # v2.Normalize(mean=[0.5], std=[0.25])
    v2.RandomVerticalFlip(p=1.0)
])

In [10]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-GRAYSCALE-NO-NORMALIZATION')
MODEL_NAME = "LSTM"
DATASET = 'fog-detection'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 1
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-GRAYSCALE-NO-NORMALIZATION/LSTM-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 313 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
313 K     Trainable params
0        

Epoch 19: 100%|██████████| 12/12 [00:07<00:00,  1.64it/s, v_num=1]         


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.11it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6132075190544128     │
│          test_f1          │    0.7602339386940002     │
│      test_precision       │    0.6132075190544128     │
│        test_recall        │            1.0            │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.517241358757019     │
│          test_f1          │    0.6818181872367859     │
│      test_precision       │     0.517241358757019     │
│        test_recall        │            1.0            │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │            0.5            │
│          test_f1          │    0.6666666865348816     │
│      test_precision       │            0.5            │
│        test_recall        │            1.0            │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6132,0.7602,0.6132,1.0
fog-or-smog,0.5172,0.6818,0.5172,1.0
foggy-cityscapes,0.5,0.6667,0.5,1.0


In [11]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-GRAYSCALE-NO-NORMALIZATION')
MODEL_NAME = "LSTM"
DATASET = 'fog-or-smog'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 1
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-GRAYSCALE-NO-NORMALIZATION/LSTM-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 313 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
313 K     Trainable params
0         N

Epoch 8: 100%|██████████| 52/52 [00:07<00:00,  7.41it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.21it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6320754885673523     │
│          test_f1          │    0.6666666865348816     │
│      test_precision       │           0.75            │
│        test_recall        │    0.6000000238418579     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7995689511299133     │
│          test_f1          │    0.8201160430908203     │
│      test_precision       │    0.7653429508209229     │
│        test_recall        │    0.8833333253860474     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6565656661987305     │
│          test_f1          │    0.5802469253540039     │
│      test_precision       │    0.7460317611694336     │
│        test_recall        │    0.4747474789619446     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6321,0.6667,0.75,0.6
fog-or-smog,0.7996,0.8201,0.7653,0.8833
foggy-cityscapes,0.6566,0.5802,0.746,0.4747


In [12]:
SAVE_DIR = Path('runs/classify/IDEA-LSTM-ROW-GRAYSCALE-NO-NORMALIZATION')
MODEL_NAME = "LSTM"
DATASET = 'foggy-cityscapes'
VERSION = 1

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_CHANNELS = 1
HIDDEN_SIZE = 128
NUM_LSTM_LAYERS = 2
NUM_CLASSES = 2

trainer = L.Trainer(
    max_epochs=40,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = LSTMClassifier(
    image_width=IMAGE_WIDTH,
    image_height=IMAGE_HEIGHT,
    image_channels=IMAGE_CHANNELS,
    hidden_size=HIDDEN_SIZE,
    num_lstm_layers=NUM_LSTM_LAYERS,
    num_classes=NUM_CLASSES
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = LSTMClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/IDEA-LSTM-ROW-GRAYSCALE-NO-NORMALIZATION/LSTM-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | LSTM             | 313 K  | train
1 | fc                 | Linear           | 258    | train
2 | criterion          | CrossEntropyLoss | 0      | train
3 | train_metrics      | MetricCollection | 0      | train
4 | validation_metrics | MetricCollection | 0      | train
5 | test_metrics       | MetricCollection | 0      | train
----------------------------------------------------------------
313 K     Trainable params
0     

Epoch 18: 100%|██████████| 22/22 [00:15<00:00,  1.43it/s, v_num=1]         


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:03<00:00,  1.24it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6698113083839417     │
│          test_f1          │     0.732824444770813     │
│      test_precision       │    0.7272727489471436     │
│        test_recall        │    0.7384615540504456     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7349137663841248     │
│          test_f1          │     0.787564754486084     │
│      test_precision       │    0.6725663542747498     │
│        test_recall        │     0.949999988079071     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:12<00:00,  0.55it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.631313145160675     │
│          test_f1          │    0.6893616914749146     │
│      test_precision       │     0.595588207244873     │
│        test_recall        │    0.8181818127632141     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6698,0.7328,0.7273,0.7385
fog-or-smog,0.7349,0.7876,0.6726,0.95
foggy-cityscapes,0.6313,0.6894,0.5956,0.8182
